<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Winter 2024 - Mini-Assembler</h1></center>
<center><h1>Due: Sunday, March 17th, 2024 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter):  <br>
    Your student ID (submitter):
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Assembler</h1></center>
<br>


The objective of this project is to implement software to assemble a (very short and non-repetitive) genome from synthetic reads. Here we assume that reads have no sequencing errors. (Credit: Ben Langmead)


**Step 1:** Let's get the reads in FASTA format. The reads come from the same synthetic genome and each is 100 bp long. For simplicity, these reads don't have any quality values. 

In [1]:
!wget http://www.cs.ucr.edu/~stelo/cs144winter24/data/reads.fa

--2024-02-14 12:06:31--  http://www.cs.ucr.edu/~stelo/cs144winter24/data/reads.fa
Resolving www.cs.ucr.edu (www.cs.ucr.edu)... 169.235.30.15
Connecting to www.cs.ucr.edu (www.cs.ucr.edu)|169.235.30.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545000 (532K)
Saving to: 'reads.fa.3'

     0K .......... .......... .......... .......... ..........  9% 1.33M 0s
    50K .......... .......... .......... .......... .......... 18% 3.59M 0s
   100K .......... .......... .......... .......... .......... 28% 5.66M 0s
   150K .......... .......... .......... .......... .......... 37% 23.5M 0s
   200K .......... .......... .......... .......... .......... 46% 2.72M 0s
   250K .......... .......... .......... .......... .......... 56% 40.1M 0s
   300K .......... .......... .......... .......... .......... 65% 54.7M 0s
   350K .......... .......... .......... .......... .......... 75% 67.9M 0s
   400K .......... .......... .......... .......... .......... 84% 4.20M 0s
   

**Step 2:** Write code to find each read's *best buddy to the right* with a time-efficient algorithm. For each read **A**, find the other read **B** that has the **longest** suffix/prefix exact match with **A**. In other words, if a suffix of **A** matches a prefix of **B**, **B** is **A**'s *best buddy* to the right. **However**, if there is a tie, or if the longest suffix/prefix match is less than 40 nucleotides long, then **A** *has no best buddy to the right*.  For each read, your program should output either (a) nothing, if there is no best buddy to the right, or (b) a single, space-separated line with the IDs of **A** and **B** and the length of the overlap, like this:

    0255/2 2065/1 88

This indicates a 88 bp suffix of the read with ID **0255/2** that matches a prefix of the read with ID **2065/1**.  Because of how we defined best buddy, it also means no other read besides **2065/1** has a prefix of 88bp or longer that is also a suffix of read **0255/2**.  A corrolary of this is that a particular read ID should appear in the first column of your program's output at most once.  Also, since we require the overlap to be at least 40 bases long, the last column should not contain any number smaller than 40.

Notes:
* You can assume all reads are error-free and from the forward strand.  In other words, you do **not** need to consider sequencing errors or reverse complements
* The order of the output lines is not important
* Reading/parsing the FASTA files can be done with BioPython, or by borrowing code from <A HREF="https://nbviewer.jupyter.org/github/BenLangmead/comp-genomics-class/blob/master/notebooks/FASTA.ipynb">this</A> if you want your own custom solution

Important: Try to avoid comparing every read to every other read when looking for suffix/prefix matches. This can be achieved by grouping "similar" reads, that is reads that share common k-mers.  For instance, you can build a dictionary where the keys are k-mers and the values are sets containing the names of all reads containing that k-mer.  Since you are looking for overlaps of length at least 40, you only need to compare reads if they have at least 1 40-mer in common.

You are allowed to use any Python package that you want. You should be able install packages using `!pip install package`. Remember that it is mandatory to acknowledge sources.

In [7]:

# Converts fasta file into 
# key : name
# value : 100 length nucleotide read
def convertFastaToDict(fastaFilePath : str) -> dict[str, str]:

        nameToSequence : dict = {}



        # Get file contents

        with open(fastaFilePath, 'r') as file:



            line1 = file.readline()
        

            while line1:

                line2 = file.readline()
                
            
                if not line2:
                    break 
                

                line1 = line1.strip()
                name = line1[1:]
                sequence =  line2.strip()
                
                
                
                # Add to dictionary

                nameToSequence[name] = sequence

                line1 = file.readline()
        
        return nameToSequence


# Test

# fastaDict = convertFastaToDict(fastaFilePath="reads.fa")

# print(fastaDict)

In [8]:


# Compares 2 strings and finds the overlap of the left's suffix with the right's prefix. It outputs this overlap string

def returnBBRSequence(mainSequence: str, rightSequence: str) -> str:
    # Initialize the length of the longest common sequence to 0
    longest_common_sequence = ""
    

    for i in range(1, min(len(mainSequence), len(rightSequence)) + 1):

        suffix = mainSequence[-i:]
        prefix = rightSequence[:i]
        

        if suffix == prefix:
            longest_common_sequence = suffix 
    

    return longest_common_sequence


# Test
# Should print 88

# mainString = "GCCTGTAGCTTAATCAACTGGACGCTGTTCGTGAAGCACCTCTCACATGATGGTTCTGCATGAAGGCTTCGCCTGCAACGAGGTGCTGCAGAAATCAGGA"
# rightString = "ATCAACTGGACGCTGTTCGTGAAGCACCTCTCACATGATGGTTCTGCATGAAGGCTTCGCCTGCAACGAGGTGCTGCAGAAATCAGGAAATCGAAGGCGA"
# print(len(returnBBRSequence(mainString, rightString)))



In [11]:
def find_kmers(string, k):
    kmers = []
    for i in range(len(string) - k + 1):
        kmers.append(string[i:i+k])
    return kmers

def kmerToNames(nameToSeq : dict) -> dict[str, set]:


    kmerToNames : dict[str,set] = dict()


    # For each name, we need to check its 40-mer prefix and 40-mer suffix

    for name,sequence in nameToSeq.items():

        kmers : list = find_kmers(sequence,40)

        # print(len(kmers))  Should be 61

        for eachKmer in kmers:

            kmerToNames[eachKmer].add(name)

        


fastaDict = convertFastaToDict(fastaFilePath="reads.fa")
kmerToNames(fastaDict)


61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
6

**Step 3:** Write a program that takes the output of the overlap program from Step 2 and creates **uni**quely assemblable con**tigs** (unitigs), using the algorithm described below.

We already determined each read's best buddy *to the right*.  I'll abbreviate this as bbr.  We did not attempt to compute each read's best buddy *to the left* (bbl), but we can infer it from the bbrs.  Consider the following output:

    A B 60
    E A 40
    C B 70
    D C 40

**A**'s bbr is **B**.  But **B**'s bbl is **C**, *not* **A**!  Your program should form unitigs by joining together two reads **X** and **Y** if they are *mutual best buddies*.  **X** and **Y** are mutual best buddies if **X**'s bbr is **Y** **and** **Y**'s bbl is **X**, or vice versa.  In this example, we would join **D**, **C**, and **B** into a single unitig (and in that order), and would join reads **E** and **A** into a single unitig (also in that order).

Your program's output should consist of several entries like the following, with one entry per unitig:

    START UNITIG 1 D
      C 40
      B 70
    END UNITIG 1
    START UNITIG 2 E
      A 40
    END UNITIG 2

The first entry represents a unitig with ID **1** consisting of 3 reads.  The first (leftmost) read is **D**.  The second read, **C**, has a **40** nt prefix that is a suffix of the previous read (**D**).  The third (rightmost) read in the contig (**B**) has a **70** bp prefix that is a suffix of the previous read (**C**).

Each read should be contained in exactly one unitig.  The order of unitigs in the file is not important, but the unitig IDs should be integers and assigned in ascending order.

You are allowed to use any Python package that you want but it is mandatory to acknowledge sources.

**Hint:** the correct solution for this particular input consists of exactly 4 unitigs

In [10]:
## YOUR CODE HERE
## you are allowed to use any python library you want

**Step 4**: Write a program that produces the final genome by processing the unitig file. This requires that you compare the unitigs to each other, think about what order they must go in, and then put them together accordingly. The final genome should be written in FASTA format (again, use BioPython or write your own code). You are allowed to use any Python package that you want, but it is mandatory to acknowledge sources.

**Hints:**

* the correct genome is 7959 nucleotides long
* to learn how the unitigs should go together, try overlapping them with each other
* a unitig can be used more than once in the solution

In [11]:
## YOUR CODE HERE
## you are allowed to use any python library you want